In [6]:
import numpy as np
import pandas as pd
import pickle
import statsmodels.api as sm
from linearmodels.panel import PanelOLS
from linearmodels.panel import compare

import FixedEffectModelPyHDFE.api as FEM
from fixedeffect.fe import fixedeffect, did, getfe

from fixedeffect.fe import fixedeffect # for 3-way fixed https://pypi.org/project/FixedEffectModel/
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib

import seaborn as sns

matplotlib.rcParams['font.family'] = 'serif'

In [8]:
data_dir = '/home/simon/Documents/Bodies/data/done_dfs/'

with open(f'{data_dir}bodies_df_2022_v1_4.pkl', 'rb') as file:
    bodies_df = pickle.load(file)

In [9]:
# if you only want more robust observatino:

bodies_df = bodies_df[(bodies_df['distance_days'] <= 48) | (bodies_df['location_annot'] == 1)].copy() # only locations you are certain of
bodies_df = bodies_df[bodies_df['city'] != 'nan']

bodies_df = bodies_df[bodies_df['year'] >= 2003]
lwd = [294, 299, 300] # likely wrong dates/months. 
bodies_df = bodies_df[~bodies_df['month_id'].isin(lwd)].copy()

# indtil det er styr på det her:
bodies_df = bodies_df[bodies_df['publication'] != 'Published']

In [10]:
#bodies_df[['Published', 'Raw', 'Submitted']] = pd.get_dummies(bodies_df[['publication']])
bodies_df[['Raw', 'Submitted']] = pd.get_dummies(bodies_df[['publication']])

bodies_df.drop(columns= 'publication', inplace= True)

# for FE
bodies_df.set_index(['gid', 'month_id'], inplace = True)

In [19]:
bodies_df = bodies_df[bodies_df['person_mean']>0]

bodies_df['lgarments'] = np.log(bodies_df['religiousGarmentFemale_mean'] + 1)
bodies_df['lfemale'] = np.log(bodies_df['female_mean'] + 1)

bodies_df['luniform'] = np.log(bodies_df['uniformed_mean'] + 1)

bodies_df['female_dummy'] = (bodies_df['female_mean'] > 0)*1
bodies_df['garment_dummy'] = (bodies_df['religiousGarmentFemale_mean'] > 0)*1

bodies_df['pp'] = bodies_df['public_mean'] - bodies_df['privat_mean']
bodies_df['log_best_public'] = bodies_df['log_best'] * bodies_df['pp']


# public_df = bodies_df[(bodies_df['pp']>0)&(bodies_df['person_mean']>0)]
# privat_df = bodies_df[(bodies_df['pp']<0)&(bodies_df['person_mean']>0)]

public_df = bodies_df[bodies_df['pp']>0]
privat_df = bodies_df[bodies_df['pp']<0]



In [12]:
# NAJAF
# bodies_df = bodies_df[bodies_df['city'] != 'Najaf'] - dues not change any major conclusions but some significane is lost...

In [13]:
bodies_df['religiousGarmentFemale_mean'].describe()

count    123826.000000
mean          0.140701
std           0.347154
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           7.500000
Name: religiousGarmentFemale_mean, dtype: float64

## As conflict intensifies we will see relatively less women overall

In [14]:
df_demean = bodies_df.copy()

df_demean['Mean_female_byPer'] = df_demean.groupby('person_mean').lfemale.transform(np.mean)
df_demean['Mean_public_byPer'] = df_demean.groupby('person_mean').pp.transform(np.mean)


#df_demean['Mean_female_byPer'] = df_demean.groupby('person_mean').female_mean.transform(np.mean)
df_demean['Mean_log_best_byPer'] = df_demean.groupby('person_mean').log_best.transform(np.mean)
#df_demean['Mean_public_byPer'] = df_demean.groupby('person_mean').public_mean.transform(np.mean)
df_demean['Mean_rural_byPer'] = df_demean.groupby('person_mean').rural_mean.transform(np.mean)
df_demean['Mean_nlights_byPer'] = df_demean.groupby('person_mean').nlights_calib_mean.transform(np.mean)
df_demean['Mean_submitted_byPer'] = df_demean.groupby('person_mean').Submitted.transform(np.mean)
df_demean['Mean_uniformed_byPer'] = df_demean.groupby('person_mean').uniformed_mean.transform(np.mean)


df_demean["female_demean"] = df_demean["lfemale"] - df_demean['Mean_female_byPer']
df_demean["public_demean"] = df_demean["pp"] - df_demean['Mean_public_byPer']

#df_demean["female_demean"] = df_demean["female_mean"] - df_demean['Mean_female_byPer']
df_demean["log_best_demean"] = df_demean["log_best"] - df_demean['Mean_log_best_byPer']
#df_demean["public_demean"] = df_demean["public_mean"] - df_demean['Mean_public_byPer']
df_demean["rural_demean"] = df_demean["rural_mean"] - df_demean['Mean_rural_byPer']
df_demean["nlights_demean"] = df_demean["nlights_calib_mean"] - df_demean['Mean_nlights_byPer']
df_demean["submitted_demean"] = df_demean["Submitted"] - df_demean['Mean_submitted_byPer']
df_demean["uniformed_demean"] = df_demean["uniformed_mean"] - df_demean['Mean_uniformed_byPer']



In [15]:
# PERSON FIXED EFFECTS

X = df_demean.loc[:,['log_best_demean']] 
#X = sm.add_constant(X)
y = df_demean.loc[:, 'female_demean']

m = PanelOLS(dependent=y, exog=X, entity_effects=True, time_effects=True)
model1 = m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

X = df_demean.loc[:,[ 'log_best_demean', 'public_demean']] 
#X = sm.add_constant(X)

y = df_demean.loc[:, 'female_demean']

m = PanelOLS(dependent=y, exog=X, entity_effects=True, time_effects=True)
model2 = m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

X = df_demean.loc[:,[ 'log_best_demean', 'public_demean', 'rural_demean']] 
#X = sm.add_constant(X)

y = df_demean.loc[:, 'female_demean']

m = PanelOLS(dependent=y, exog=X, entity_effects=True, time_effects=True)
model3 = m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

X = df_demean.loc[:,[ 'log_best_demean', 'public_demean', 'rural_demean', 'nlights_demean']] 
#X = sm.add_constant(X)

y = df_demean.loc[:, 'female_demean']

m = PanelOLS(dependent=y, exog=X, entity_effects=True, time_effects=True)
model4 = m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

X = df_demean.loc[:,[ 'log_best_demean', 'public_demean', 'rural_demean', 'nlights_demean', 'submitted_demean']] 
#X = sm.add_constant(X)

y = df_demean.loc[:, 'female_demean']

m = PanelOLS(dependent=y, exog=X, entity_effects=True, time_effects=True)
model5 = m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

X = df_demean.loc[:,[ 'log_best_demean', 'public_demean', 'rural_demean', 'nlights_demean', 'submitted_demean', 'uniformed_demean']] 
#X = sm.add_constant(X)

y = df_demean.loc[:, 'female_demean']

m = PanelOLS(dependent=y, exog=X, entity_effects=True, time_effects=True)
model6 = m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

print(compare({"model1": model1, "model2": model2, "model3": model3, "model4": model4, "model5": model5, "model6": model6}, precision = 'std_errors', stars = True))

                                                          Model Comparison                                                         
                                   model1            model2            model3            model4            model5            model6
-----------------------------------------------------------------------------------------------------------------------------------
Dep. Variable               female_demean     female_demean     female_demean     female_demean     female_demean     female_demean
Estimator                        PanelOLS          PanelOLS          PanelOLS          PanelOLS          PanelOLS          PanelOLS
No. Observations                   123826            123826            123826            123826            123826            123826
Cov. Est.                       Clustered         Clustered         Clustered         Clustered         Clustered         Clustered
R-squared                          0.0001            0.1035            0.172

In [16]:
bodies_df['rural_mean'].describe()

count    123826.000000
mean         -0.091655
std           0.374089
min          -1.127627
25%          -0.366175
50%          -0.162569
75%           0.126683
max           1.474247
Name: rural_mean, dtype: float64

In [17]:
np.percentile(bodies_df['rural_mean'],15)

-0.4536036401987076

In [18]:
np.percentile(bodies_df['rural_mean'],)

TypeError: _percentile_dispatcher() missing 1 required positional argument: 'q'

In [ ]:
bodies_df[bodies_df['rural_mean'] >= 1].shape[0] / bodies_df.shape[0]

0.008552323421575437

# As conflict intensifies we will see relatively more women in religious garments

In [ ]:
#df_demean = bodies_df[bodies_df['person_mean']>0].copy()
df_demean = bodies_df.copy()

df_demean['Mean_garments_byFem'] = df_demean.groupby('female_mean').lgarments.transform(np.mean)
df_demean['Mean_public_byFem'] = df_demean.groupby('female_mean').pp.transform(np.mean)



#df_demean['Mean_garments_byFem'] = df_demean.groupby('female_mean').religiousGarmentFemale_mean.transform(np.mean)
df_demean['Mean_log_best_byFem'] = df_demean.groupby('female_mean').log_best.transform(np.mean)
#df_demean['Mean_public_byFem'] = df_demean.groupby('female_mean').public_mean.transform(np.mean)
df_demean['Mean_rural_byFem'] = df_demean.groupby('female_mean').rural_mean.transform(np.mean)
df_demean['Mean_nlights_byFem'] = df_demean.groupby('female_mean').nlights_calib_mean.transform(np.mean)
df_demean['Mean_submitted_byFem'] = df_demean.groupby('female_mean').Submitted.transform(np.mean)
df_demean['Mean_uniformed_byFem'] = df_demean.groupby('female_mean').uniformed_mean.transform(np.mean)


df_demean["garments_demean"] = df_demean["lgarments"] - df_demean['Mean_garments_byFem']
df_demean["public_demean"] = df_demean["pp"] - df_demean['Mean_public_byFem']


#df_demean["garments_demean"] = df_demean["religiousGarmentFemale_mean"] - df_demean['Mean_garments_byFem']
df_demean["log_best_demean"] = df_demean["log_best"] - df_demean['Mean_log_best_byFem']
#df_demean["public_demean"] = df_demean["public_mean"] - df_demean['Mean_public_byFem']
df_demean["rural_demean"] = df_demean["rural_mean"] - df_demean['Mean_rural_byFem']
df_demean["nlights_demean"] = df_demean["nlights_calib_mean"] - df_demean['Mean_nlights_byFem']
df_demean["submitted_demean"] = df_demean["Submitted"] - df_demean['Mean_submitted_byFem']
df_demean["uniformed_demean"] = df_demean["uniformed_mean"] - df_demean['Mean_uniformed_byFem']



In [ ]:
# FEMALE FIXED EFFECTS

X = df_demean.loc[:,['log_best_demean']] 
#X = sm.add_constant(X)
y = df_demean.loc[:, 'garments_demean']

m = PanelOLS(dependent=y, exog=X, entity_effects=True, time_effects=True)
model1 = m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

X = df_demean.loc[:,[ 'log_best_demean', 'public_demean']] 
#X = sm.add_constant(X)

y = df_demean.loc[:, 'garments_demean']

m = PanelOLS(dependent=y, exog=X, entity_effects=True, time_effects=True)
model2 = m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

X = df_demean.loc[:,[ 'log_best_demean', 'public_demean', 'rural_demean']] 
#X = sm.add_constant(X)

y = df_demean.loc[:, 'garments_demean']

m = PanelOLS(dependent=y, exog=X, entity_effects=True, time_effects=True)
model3 = m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

X = df_demean.loc[:,[ 'log_best_demean', 'public_demean', 'rural_demean', 'nlights_demean']] 
#X = sm.add_constant(X)

y = df_demean.loc[:, 'garments_demean']

m = PanelOLS(dependent=y, exog=X, entity_effects=True, time_effects=True)
model4 = m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

X = df_demean.loc[:,[ 'log_best_demean', 'public_demean', 'rural_demean', 'nlights_demean', 'submitted_demean']] 
#X = sm.add_constant(X)

y = df_demean.loc[:, 'garments_demean']

m = PanelOLS(dependent=y, exog=X, entity_effects=True, time_effects=True)
model5 = m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

X = df_demean.loc[:,[ 'log_best_demean', 'public_demean', 'rural_demean', 'nlights_demean', 'submitted_demean', 'uniformed_demean']] 
#X = sm.add_constant(X)

y = df_demean.loc[:, 'garments_demean']

m = PanelOLS(dependent=y, exog=X, entity_effects=True, time_effects=True)
model6 = m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

print(compare({"model1": model1, "model2": model2, "model3": model3, "model4": model4, "model5": model5, "model6": model6}, precision = 'std_errors', stars = True))

                                                                Model Comparison                                                               
                                     model1              model2              model3              model4              model5              model6
-----------------------------------------------------------------------------------------------------------------------------------------------
Dep. Variable               garments_demean     garments_demean     garments_demean     garments_demean     garments_demean     garments_demean
Estimator                          PanelOLS            PanelOLS            PanelOLS            PanelOLS            PanelOLS            PanelOLS
No. Observations                     123826              123826              123826              123826              123826              123826
Cov. Est.                         Clustered           Clustered           Clustered           Clustered           Clustered           Cl

### The decreased presence of women will be more prevailing in public scenes
### The decreased presence of women will be less prevailing in private scenes

In [ ]:
# but why not just control?

df_demean_pub = public_df.copy()
df_demean_pri = privat_df.copy()
df_demean = bodies_df.copy() 

df_demean_pub['Mean_female_byPer'] = df_demean_pub.groupby('person_mean').lfemale.transform(np.mean)
df_demean_pri['Mean_female_byPer'] = df_demean_pri.groupby('person_mean').lfemale.transform(np.mean)
df_demean['Mean_female_byPer'] = df_demean.groupby('person_mean').lfemale.transform(np.mean)

# df_demean_pub['Mean_female_byPer'] = df_demean_pub.groupby('person_mean').female_mean.transform(np.mean)
#df_demean_pri['Mean_female_byPer'] = df_demean_pri.groupby('person_mean').female_mean.transform(np.mean)
#df_demean['Mean_female_byPer'] = df_demean.groupby('person_mean').female_mean.transform(np.mean)

df_demean_pub['Mean_log_best_byPer'] = df_demean_pub.groupby('person_mean').log_best.transform(np.mean)
df_demean_pri['Mean_log_best_byPer']  = df_demean_pri.groupby('person_mean').log_best.transform(np.mean)
df_demean['Mean_log_best_byPer']  = df_demean.groupby('person_mean').log_best.transform(np.mean)

df_demean_pub['Mean_public_byPer'] = df_demean_pub.groupby('person_mean').public_mean.transform(np.mean)
df_demean_pri['Mean_public_byPer'] = df_demean_pri.groupby('person_mean').public_mean.transform(np.mean)
df_demean['Mean_public_byPer'] = df_demean.groupby('person_mean').public_mean.transform(np.mean)

df_demean_pub['Mean_lbp_byPer'] = df_demean_pub.groupby('person_mean').log_best_public.transform(np.mean)
df_demean_pri['Mean_lbp_byPer'] = df_demean_pri.groupby('person_mean').log_best_public.transform(np.mean)
df_demean['Mean_lbp_byPer'] = df_demean.groupby('person_mean').log_best_public.transform(np.mean)

# ----------------------------------------------------------------------------------------

df_demean_pub['female_demean'] = df_demean_pub["lfemale"] - df_demean_pub['Mean_female_byPer']
df_demean_pri['female_demean'] =  df_demean_pri["lfemale"] - df_demean_pri['Mean_female_byPer']
df_demean['female_demean'] =  df_demean["lfemale"] - df_demean['Mean_female_byPer']


#df_demean_pub['female_demean'] = df_demean_pub["female_mean"] - df_demean_pub['Mean_female_byPer']
# df_demean_pri['female_demean'] =  df_demean_pri["female_mean"] - df_demean_pri['Mean_female_byPer']
# df_demean['female_demean'] =  df_demean["female_mean"] - df_demean['Mean_female_byPer']

df_demean_pub['log_best_demean']  = df_demean_pub["log_best"] - df_demean_pub['Mean_log_best_byPer']
df_demean_pri['log_best_demean'] = df_demean_pri["log_best"] - df_demean_pri['Mean_log_best_byPer']
df_demean['log_best_demean'] = df_demean["log_best"] - df_demean['Mean_log_best_byPer']

df_demean_pub['public_demean'] = df_demean_pub["public_mean"] - df_demean_pub['Mean_public_byPer']
df_demean_pri['public_demean'] = df_demean_pri["public_mean"] - df_demean_pri['Mean_public_byPer']
df_demean['public_demean'] = df_demean["public_mean"] - df_demean['Mean_public_byPer']

df_demean_pub['lbp_demean'] = df_demean_pub["log_best_public"] - df_demean_pub['Mean_lbp_byPer']
df_demean_pri['lbp_demean'] = df_demean_pri["log_best_public"] - df_demean_pri['Mean_lbp_byPer']
df_demean['lbp_demean'] = df_demean["log_best_public"] - df_demean['Mean_lbp_byPer']

In [ ]:
# But why not person fixed effects?
# And why not subset the df to person > 0 as below

X = df_demean_pub.loc[:,['log_best_demean']] 
#X = sm.add_constant(X)
y = df_demean_pub.loc[:, 'female_demean']

m = PanelOLS(dependent=y, exog=X, entity_effects=True, time_effects=True)
model1 = m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

X = df_demean_pri.loc[:,[ 'log_best_demean']] 
#X = sm.add_constant(X)

y = df_demean_pri.loc[:, 'female_demean']

m = PanelOLS(dependent=y, exog=X, entity_effects=True, time_effects=True)
model2 = m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

X = df_demean.loc[:,[ 'log_best_demean', 'public_demean', 'lbp_demean']] 
#X = sm.add_constant(X)

y = df_demean.loc[:, 'female_demean']

m = PanelOLS(dependent=y, exog=X, entity_effects=True, time_effects=True)
model3 = m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)



print(compare({"model1": model1, "model2": model2, "model3": model3}, precision = 'std_errors', stars = True))

                               Model Comparison                              
                                   model1            model2            model3
-----------------------------------------------------------------------------
Dep. Variable               female_demean     female_demean     female_demean
Estimator                        PanelOLS          PanelOLS          PanelOLS
No. Observations                    55950             67876            123826
Cov. Est.                       Clustered         Clustered         Clustered
R-squared                          0.0014         8.759e-06            0.1018
R-Squared (Within)                -0.0277           -0.0026            0.0907
R-Squared (Between)               -0.1494           -0.0407            0.0458
R-Squared (Overall)               -0.0392           -0.0035            0.0928
F-statistic                        76.141            0.5940            4674.2
P-value (F-stat)                   0.0000            0.4409     

In [ ]:
# but why not just control?

df_demean_pub = public_df.copy()
df_demean_pri = privat_df.copy()
df_demean = bodies_df.copy() 

df_demean_pub['Mean_female_byPer'] = df_demean_pub.groupby('person_mean').lfemale.transform(np.mean)
df_demean_pri['Mean_female_byPer'] = df_demean_pri.groupby('person_mean').lfemale.transform(np.mean)
df_demean['Mean_female_byPer'] = df_demean.groupby('person_mean').lfemale.transform(np.mean)

# df_demean_pub['Mean_female_byPer'] = df_demean_pub.groupby('person_mean').female_mean.transform(np.mean)
#df_demean_pri['Mean_female_byPer'] = df_demean_pri.groupby('person_mean').female_mean.transform(np.mean)
#df_demean['Mean_female_byPer'] = df_demean.groupby('person_mean').female_mean.transform(np.mean)

df_demean_pub['Mean_log_best_byPer'] = df_demean_pub.groupby('person_mean').log_best.transform(np.mean)
df_demean_pri['Mean_log_best_byPer']  = df_demean_pri.groupby('person_mean').log_best.transform(np.mean)
df_demean['Mean_log_best_byPer']  = df_demean.groupby('person_mean').log_best.transform(np.mean)

df_demean_pub['Mean_public_byPer'] = df_demean_pub.groupby('person_mean').public_mean.transform(np.mean)
df_demean_pri['Mean_public_byPer'] = df_demean_pri.groupby('person_mean').public_mean.transform(np.mean)
df_demean['Mean_public_byPer'] = df_demean.groupby('person_mean').public_mean.transform(np.mean)

df_demean_pub['Mean_lbp_byPer'] = df_demean_pub.groupby('person_mean').log_best_public.transform(np.mean)
df_demean_pri['Mean_lbp_byPer'] = df_demean_pri.groupby('person_mean').log_best_public.transform(np.mean)
df_demean['Mean_lbp_byPer'] = df_demean.groupby('person_mean').log_best_public.transform(np.mean)

df_demean_pub['Mean_rural_byPer'] = df_demean_pub.groupby('person_mean').rural_mean.transform(np.mean)
df_demean_pri['Mean_rural_byPer'] = df_demean_pri.groupby('person_mean').rural_mean.transform(np.mean)
df_demean['Mean_rural_byPer'] = df_demean.groupby('person_mean').rural_mean.transform(np.mean)

# ----------------------------------------------------------------------------------------

df_demean_pub['female_demean'] = df_demean_pub["lfemale"] - df_demean_pub['Mean_female_byPer']
df_demean_pri['female_demean'] =  df_demean_pri["lfemale"] - df_demean_pri['Mean_female_byPer']
df_demean['female_demean'] =  df_demean["lfemale"] - df_demean['Mean_female_byPer']


#df_demean_pub['female_demean'] = df_demean_pub["female_mean"] - df_demean_pub['Mean_female_byPer']
# df_demean_pri['female_demean'] =  df_demean_pri["female_mean"] - df_demean_pri['Mean_female_byPer']
# df_demean['female_demean'] =  df_demean["female_mean"] - df_demean['Mean_female_byPer']

df_demean_pub['log_best_demean']  = df_demean_pub["log_best"] - df_demean_pub['Mean_log_best_byPer']
df_demean_pri['log_best_demean'] = df_demean_pri["log_best"] - df_demean_pri['Mean_log_best_byPer']
df_demean['log_best_demean'] = df_demean["log_best"] - df_demean['Mean_log_best_byPer']

df_demean_pub['public_demean'] = df_demean_pub["public_mean"] - df_demean_pub['Mean_public_byPer']
df_demean_pri['public_demean'] = df_demean_pri["public_mean"] - df_demean_pri['Mean_public_byPer']
df_demean['public_demean'] = df_demean["public_mean"] - df_demean['Mean_public_byPer']

df_demean_pub['lbp_demean'] = df_demean_pub["log_best_public"] - df_demean_pub['Mean_lbp_byPer']
df_demean_pri['lbp_demean'] = df_demean_pri["log_best_public"] - df_demean_pri['Mean_lbp_byPer']
df_demean['lbp_demean'] = df_demean["log_best_public"] - df_demean['Mean_lbp_byPer']

df_demean_pub['rural_demean'] = df_demean_pub["rural_mean"] - df_demean_pub['Mean_rural_byPer']
df_demean_pri['rural_demean'] = df_demean_pri["rural_mean"] - df_demean_pri['Mean_rural_byPer']
df_demean['rural_demean'] = df_demean["rural_mean"] - df_demean['Mean_rural_byPer']

In [ ]:
# But why not person fixed effects?
# And why not subset the df to person > 0 as below

X = df_demean_pub.loc[:,['log_best_demean', 'rural_demean']] 
#X = sm.add_constant(X)
y = df_demean_pub.loc[:, 'female_demean']

m = PanelOLS(dependent=y, exog=X, entity_effects=True, time_effects=True)
model1 = m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

X = df_demean_pri.loc[:,[ 'log_best_demean', 'rural_demean']] 
#X = sm.add_constant(X)

y = df_demean_pri.loc[:, 'female_demean']

m = PanelOLS(dependent=y, exog=X, entity_effects=True, time_effects=True)
model2 = m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

X = df_demean.loc[:,[ 'log_best_demean', 'rural_demean', 'public_demean', 'lbp_demean']] 
#X = sm.add_constant(X)

y = df_demean.loc[:, 'female_demean']

m = PanelOLS(dependent=y, exog=X, entity_effects=True, time_effects=True)
model3 = m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)



print(compare({"model1": model1, "model2": model2, "model3": model3}, precision = 'std_errors', stars = True))

                               Model Comparison                              
                                   model1            model2            model3
-----------------------------------------------------------------------------
Dep. Variable               female_demean     female_demean     female_demean
Estimator                        PanelOLS          PanelOLS          PanelOLS
No. Observations                    55950             67876            123826
Cov. Est.                       Clustered         Clustered         Clustered
R-squared                          0.1016            0.0813            0.1737
R-Squared (Within)                 0.1267            0.0814            0.1906
R-Squared (Between)               -0.0942            0.3101            0.0532
R-Squared (Overall)                0.1305            0.0848            0.2017
F-statistic                        3159.6            3001.7            6504.2
P-value (F-stat)                   0.0000            0.0000     


### The women who does appear in public spaces are more likely to where religious garments

In [ ]:
# but why not just control?

df_demean_pub = public_df.copy()
df_demean_pri = privat_df.copy()
df_demean = bodies_df.copy() 

df_demean_pub['Mean_garments_byFem'] = df_demean_pub.groupby('female_mean').lgarments.transform(np.mean)
df_demean_pri['Mean_garments_byFem'] = df_demean_pri.groupby('female_mean').lgarments.transform(np.mean)
df_demean['Mean_garments_byFem'] = df_demean.groupby('female_mean').lgarments.transform(np.mean)

#df_demean_pub['Mean_garments_byFem'] = df_demean_pub.groupby('female_mean').religiousGarmentFemale_mean.transform(np.mean)
# df_demean_pri['Mean_garments_byFem'] = df_demean_pri.groupby('female_mean').religiousGarmentFemale_mean.transform(np.mean)
# df_demean['Mean_garments_byFem'] = df_demean.groupby('female_mean').religiousGarmentFemale_mean.transform(np.mean)

df_demean_pub['Mean_log_best_byFem'] = df_demean_pub.groupby('female_mean').log_best.transform(np.mean)
df_demean_pri['Mean_log_best_byFem']  = df_demean_pri.groupby('female_mean').log_best.transform(np.mean)
df_demean['Mean_log_best_byFem']  = df_demean.groupby('female_mean').log_best.transform(np.mean)

df_demean_pub['Mean_public_byFem'] = df_demean_pub.groupby('female_mean').public_mean.transform(np.mean)
df_demean_pri['Mean_public_byFem'] = df_demean_pri.groupby('female_mean').public_mean.transform(np.mean)
df_demean['Mean_public_byFem'] = df_demean.groupby('female_mean').public_mean.transform(np.mean)

df_demean_pub['Mena_lbp_byFem'] = df_demean_pub.groupby('female_mean').log_best_public.transform(np.mean)
df_demean_pri['Mena_lbp_byFem'] = df_demean_pri.groupby('female_mean').log_best_public.transform(np.mean)
df_demean['Mena_lbp_byFem'] = df_demean.groupby('female_mean').log_best_public.transform(np.mean)



df_demean_pub['Mena_rural_byFem'] = df_demean_pub.groupby('female_mean').rural_mean.transform(np.mean)
df_demean_pri['Mena_rural_byFem'] = df_demean_pri.groupby('female_mean').rural_mean.transform(np.mean)
df_demean['Mena_rural_byFem'] = df_demean.groupby('female_mean').rural_mean.transform(np.mean)



# ----------------------------------------------------------------------------------------

df_demean_pub['garments_demean'] = df_demean_pub["lgarments"] - df_demean_pub['Mean_garments_byFem']
df_demean_pri['garments_demean'] =  df_demean_pri["lgarments"] - df_demean_pri['Mean_garments_byFem']
df_demean['garments_demean'] =  df_demean["lgarments"] - df_demean['Mean_garments_byFem']

#df_demean_pub['garments_demean'] = df_demean_pub["religiousGarmentFemale_mean"] - df_demean_pub['Mean_garments_byFem']
# df_demean_pri['garments_demean'] =  df_demean_pri["religiousGarmentFemale_mean"] - df_demean_pri['Mean_garments_byFem']
# df_demean['garments_demean'] =  df_demean["religiousGarmentFemale_mean"] - df_demean['Mean_garments_byFem']

df_demean_pub['log_best_demean']  = df_demean_pub["log_best"] - df_demean_pub['Mean_log_best_byFem']
df_demean_pri['log_best_demean'] = df_demean_pri["log_best"] - df_demean_pri['Mean_log_best_byFem']
df_demean['log_best_demean'] = df_demean["log_best"] - df_demean['Mean_log_best_byFem']

df_demean_pub['public_demean'] = df_demean_pub["public_mean"] - df_demean_pub['Mean_public_byFem']
df_demean_pri['public_demean'] = df_demean_pri["public_mean"] - df_demean_pri['Mean_public_byFem']
df_demean['public_demean'] = df_demean["public_mean"] - df_demean['Mean_public_byFem']

df_demean_pub['lbp_demean'] = df_demean_pub["log_best_public"] - df_demean_pub['Mena_lbp_byFem']
df_demean_pri['lbp_demean'] = df_demean_pri["log_best_public"] - df_demean_pri['Mena_lbp_byFem']
df_demean['lbp_demean'] = df_demean["log_best_public"] - df_demean['Mena_lbp_byFem']



df_demean_pub['rural_demean'] = df_demean_pub["rural_mean"] - df_demean_pub['Mena_rural_byFem']
df_demean_pri['rural_demean'] = df_demean_pri["rural_mean"] - df_demean_pri['Mena_rural_byFem']
df_demean['rural_demean'] = df_demean["rural_mean"] - df_demean['Mena_rural_byFem']

In [ ]:
# FEMALE FIXED EFFECTS

X = df_demean_pub.loc[:,['log_best_demean']] 
#X = sm.add_constant(X)
y = df_demean_pub.loc[:, 'garments_demean']

m = PanelOLS(dependent=y, exog=X, entity_effects=True, time_effects=True)
model1 = m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

X = df_demean_pri.loc[:,[ 'log_best_demean']] 
#X = sm.add_constant(X)

y = df_demean_pri.loc[:, 'garments_demean']

m = PanelOLS(dependent=y, exog=X, entity_effects=True, time_effects=True)
model2 = m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

X = df_demean.loc[:,[ 'log_best_demean', 'public_demean', 'lbp_demean']] 
#X = sm.add_constant(X)

y = df_demean.loc[:, 'garments_demean']

m = PanelOLS(dependent=y, exog=X, entity_effects=True, time_effects=True)
model3 = m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)



print(compare({"model1": model1, "model2": model2, "model3": model3}, precision = 'std_errors', stars = True))

                                  Model Comparison                                 
                                     model1              model2              model3
-----------------------------------------------------------------------------------
Dep. Variable               garments_demean     garments_demean     garments_demean
Estimator                          PanelOLS            PanelOLS            PanelOLS
No. Observations                      55950               67876              123826
Cov. Est.                         Clustered           Clustered           Clustered
R-squared                         1.186e-05           7.286e-05              0.0004
R-Squared (Within)               -9.979e-05             -0.0014             -0.0017
R-Squared (Between)                 -0.1431             -0.5641             -0.6692
R-Squared (Overall)                 -0.0004             -0.0038             -0.0047
F-statistic                          0.6631              4.9417             

Formuler spørgsmål til Jacob...

Obsevation: I garments split-sample modeller bruger du kun observationer med > 0 og har female fixed effects. I female split-sample modeller har du persons med som kontrol.
Spørgsmål: ville det ikke være mere konsistent hvis vi i female split samplen satte person FE og evt kun brugt observationer med person > 0? Aller alkternativt bare includerede female som kontrol i split-sample modellen?




### perhaps you should try and drop all obs from Najaf and see whant happens?

# Auxillary test

In [21]:
df_demean = bodies_df.copy()

df_demean['Mean_female_byPer'] = df_demean.groupby('person_mean').lfemale.transform(np.mean)
df_demean['Mean_public_byPer'] = df_demean.groupby('person_mean').pp.transform(np.mean)


df_demean['Mean_log_best_byPer'] = df_demean.groupby('person_mean').log_best.transform(np.mean)
df_demean['Mean_rural_byPer'] = df_demean.groupby('person_mean').rural_mean.transform(np.mean)
df_demean['Mean_nlights_byPer'] = df_demean.groupby('person_mean').nlights_calib_mean.transform(np.mean)
df_demean['Mean_submitted_byPer'] = df_demean.groupby('person_mean').Submitted.transform(np.mean)
df_demean['Mean_uniformed_byPer'] = df_demean.groupby('person_mean').uniformed_mean.transform(np.mean)


df_demean["female_demean"] = df_demean["lfemale"] - df_demean['Mean_female_byPer']
df_demean["public_demean"] = df_demean["pp"] - df_demean['Mean_public_byPer']

df_demean["log_best_demean"] = df_demean["log_best"] - df_demean['Mean_log_best_byPer']
df_demean["rural_demean"] = df_demean["rural_mean"] - df_demean['Mean_rural_byPer']
df_demean["nlights_demean"] = df_demean["nlights_calib_mean"] - df_demean['Mean_nlights_byPer']
df_demean["submitted_demean"] = df_demean["Submitted"] - df_demean['Mean_submitted_byPer']
df_demean["uniformed_demean"] = df_demean["uniformed_mean"] - df_demean['Mean_uniformed_byPer']



In [23]:
# PERSON FIXED EFFECTS

X = df_demean.loc[:,['log_best_demean']] 
#X = sm.add_constant(X)
y = df_demean.loc[:, 'uniformed_demean']

m = PanelOLS(dependent=y, exog=X, entity_effects=True, time_effects=True)
model1 = m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

X = df_demean.loc[:,[ 'log_best_demean', 'public_demean']] 
#X = sm.add_constant(X)

y = df_demean.loc[:, 'uniformed_demean']

m = PanelOLS(dependent=y, exog=X, entity_effects=True, time_effects=True)
model2 = m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

X = df_demean.loc[:,[ 'log_best_demean', 'public_demean', 'rural_demean']] 
#X = sm.add_constant(X)

y = df_demean.loc[:, 'uniformed_demean']

m = PanelOLS(dependent=y, exog=X, entity_effects=True, time_effects=True)
model3 = m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

X = df_demean.loc[:,[ 'log_best_demean', 'public_demean', 'rural_demean', 'nlights_demean']] 
#X = sm.add_constant(X)

y = df_demean.loc[:, 'uniformed_demean']

m = PanelOLS(dependent=y, exog=X, entity_effects=True, time_effects=True)
model4 = m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

X = df_demean.loc[:,[ 'log_best_demean', 'public_demean', 'rural_demean', 'nlights_demean', 'submitted_demean']] 
#X = sm.add_constant(X)

y = df_demean.loc[:, 'uniformed_demean']

m = PanelOLS(dependent=y, exog=X, entity_effects=True, time_effects=True)
model5 = m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

X = df_demean.loc[:,[ 'log_best_demean', 'public_demean', 'rural_demean', 'nlights_demean', 'submitted_demean', 'female_demean']] 
#X = sm.add_constant(X)

y = df_demean.loc[:, 'uniformed_demean']

m = PanelOLS(dependent=y, exog=X, entity_effects=True, time_effects=True)
model6 = m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

print(compare({"model1": model1, "model2": model2, "model3": model3, "model4": model4, "model5": model5, "model6": model6}, precision = 'std_errors', stars = True))

                                                                   Model Comparison                                                                  
                                      model1               model2               model3               model4               model5               model6
-----------------------------------------------------------------------------------------------------------------------------------------------------
Dep. Variable               uniformed_demean     uniformed_demean     uniformed_demean     uniformed_demean     uniformed_demean     uniformed_demean
Estimator                           PanelOLS             PanelOLS             PanelOLS             PanelOLS             PanelOLS             PanelOLS
No. Observations                      123826               123826               123826               123826               123826               123826
Cov. Est.                          Clustered            Clustered            Clustered            Cl

!!!!  This actually indicates that they a avoiding solders!!!! 

In [25]:
bodies_df[bodies_df['Submitted'] == 1]

person_mean  person_median  person_fasterR50  \
gid    month_id                                                 
177569 359               1.4            1.0               2.0   
       304               1.0            1.0               1.0   
       280               2.4            2.0               2.0   
       348               4.0            4.0               4.0   
       348               6.4            6.0               7.0   
...                      ...            ...               ...   
179009 289               2.8            3.0               3.0   
       289               0.6            1.0               1.0   
       289               2.0            2.0               2.0   
       289               4.8            5.0               5.0   
       289               6.8            7.0               7.0   

                 person_fasterR101  person_fasterX101  person_retinaR50  \
gid    month_id                                                           
177569 359                     1.0                1.0               2.0   
       304                     1.0                1.0               1.0   
       280                     3.0                3.0               2.0   
       348                     4.0                4.0               4.0   
       348                     5.0                6.0               9.0   
...                            ...                ...               ...   
179009 289                     3.0                3.0               3.0   
       289                     1.0                0.0               0.0   
       289                     2.0                2.0               2.0   
       289                     5.0                5.0               5.0   
       289                     7.0                6.0               7.0   

                 person_retinaR101  child_mean  child_median  child_retinaR50  \
gid    month_id                                                                 
177569 359                     1.0         0.0           0.0              0.0   
       304                     1.0         1.0           1.0              1.0   
       280                     2.0         0.5           0.5              1.0   
       348                     4.0         2.0           2.0              3.0   
       348                     5.0         0.0           0.0              0.0   
...                            ...         ...           ...              ...   
179009 289                     2.0         1.5           1.5              1.0   
       289                     1.0         0.0           0.0              0.0   
       289                     2.0         0.0           0.0              0.0   
       289                     4.0         1.5           1.5              2.0   
       289                     7.0         0.5           0.5              1.0   

                 ...  ttime_mean  Raw  Submitted  lgarments   lfemale  \
gid    month_id  ...                                                    
177569 359       ...    58.67639    0          1   0.000000  0.405465   
       304       ...    58.67639    0          1   0.405465  0.693147   
       280       ...    58.67639    0          1   0.000000  0.405465   
       348       ...    58.67639    0          1   0.000000  1.252763   
       348       ...    58.67639    0          1   0.000000  0.405465   
...              ...         ...  ...        ...        ...       ...   
179009 289       ...   168.32170    0          1   0.000000  0.405465   
       289       ...   168.32170    0          1   0.000000  0.000000   
       289       ...   168.32170    0          1   0.000000  0.693147   
       289       ...   168.32170    0          1   0.000000  0.405465   
       289       ...   168.32170    0          1   0.000000  0.405465   

                 female_dummy  garment_dummy        pp  log_best_public  \
gid    month_id                                                           
177569 359                  1            